In [1]:
from dotenv import load_dotenv

load_dotenv('E:\\MyOnlineCourses\\ML_Projects\\dagshubrepos\\pokmn\\tokens.env')

True

### FROM ZERO TO HERO -section

#### Deep Lake VectorStore

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType


# instantiate the LLM and embeddings models
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


e:\MyOnlineCourses\ML_Projects\dagshubrepos\pokmn\.conda\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.19) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [ ]:

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "mohdnassgabr" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

###### Create a RetrievalQA chain

In [ ]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

###### create an agent that uses the RetrievalQA chain as a tool:

In [ ]:
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True)

In [ ]:
response = agent.run("When was Napoleone born?")
print(response)

In [ ]:

my_activeloop_org_id = "mohdnassgabr" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

In [ ]:
# instantiate the wrapper class for GPT3
llm = OpenAI(model="text-davinci-003", temperature=0)

# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [ ]:
response = agent.run("When was Michael Jordan born?")
print(response)

#### Agents in LangChain

To get GOGLE_API_KEY and GOOGLE_CSE_ID :

https://python.langchain.com/docs/integrations/tools/google_search

In [3]:
from langchain.llms import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
search = GoogleSearchAPIWrapper()

In [4]:
llm = OpenAI(model="text-davinci-003", temperature=0)


In [ ]:
#!pip install google-api-python-client

In [5]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

In [7]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [9]:
response = agent("What's the latest news about the KSA vision 2030?")
print(response['output'])



> Entering new  chain...
 I should search for news articles about the KSA vision 2030
Action: google-search
Action Input: KSA vision 2030 news
Observation: Vision 2030 is a unique transformative economic and social reform blueprint that is opening Saudi Arabia up to the world. Apr 25, 2016 ... King Salman said in a televised announcement that the Cabinet approved the plan, known as Vision 2030, and called on Saudis to work together ... News. #Vision 2030. GASTAT: Non-oil activities lead to a 3.9% economic growth ... HRH Crown Prince Launches Four New Special Economic Zones in Saudi Arabia. Feb 13, 2017 ... Enter a new master plan, grandly titled Vision 2030, which was unveiled with great fanfare last year. Vision 2030 is a unique transformative economic and social reform blueprint that is opening Saudi Arabia up to the world, enriching the lives of Saudi ... The Radical Remake of Saudi Arabia. follow this storythread. Contributors: Bloomberg News. Updated on May 27, 8:20 AM EDT ... A

##### Tools of langchain


In our example, two tools are being defined for use within a LangChain agent: a Google Search tool and a Language Model tool acting specifically as a text summarizer.

In [10]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [11]:
llm = OpenAI(model="text-davinci-003", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [12]:
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

In [13]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [14]:
response = agent("What's the latest news about the climate? Then please summarize the results.")
print(response['output'])



> Entering new  chain...
 I should search for recent news about the climate and then summarize the results.
Action: Search
Action Input: Latest news about the climate
Observation: Climate in focus · The new normal - why this summer has been so very hot · Earth in uncharted waters as climate records tumble · UK no longer a world leader on ... Climate change weaves through daily lives: the global politics of global warming, the business impact of greenhouse gas and carbon emissions and the ... Vital Signs of the Planet: Global Climate Change and Global Warming. Current news and data streams about global warming and climate change from NASA. Latest Climate crisis news, comment and analysis from the Guardian, the world's leading liberal voice. Latest news and updates on climate change and the most critical issues affecting the environment globally. A digest of the most pressing climate-related news, released every Tuesday and Friday. Get Today's Climate. Breaking ... Read science article

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ib3ti2f5751AsV2Xr7sXIsan on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-ib3ti2f5751AsV2Xr7sXIsan on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the final answer.
Final Answer: Climate change is having a significant impact on the environment and daily lives, and many projects need to be completed by 2023 in order to keep warming below a certain level.

> Finished chain.
Climate change is having a significant impact on the environment and daily lives, and many projects need to be completed by 2023 in order to keep warming below a certain level.
